In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("./data/Casino_Gaming_Data_new.csv")
df


,Fiscal Year,Month Ending,Licensee,Wagers,Patron Winnings,Cancelled Wagers,Online Casino Gaming Win/(Loss),Promotional Coupons or Credits Wagered (3),Promotional Deduction (4),Total Gross Gaming Revenue,Payment (5)
0,2021/22,06/30/2022 12:00:00 AM,Mohegan Tribe On-Reservation,379023,335505,5,43513,6390,6390,37123,6682
1,2021/22,06/30/2022 12:00:00 AM,MPTN On-Reservation,671843,685886,3,-14046,29041,0,-14046,0
2,2021/22,06/30/2022 12:00:00 AM,"MPI Master Wagering License CT, LLC",420056291,406814933,164727,13076631,12372142,3269158,9807473,1765345
3,2021/22,06/30/2022 12:00:00 AM,"Mohegan Digital, LLC",254393582,246515856,11645,7866081,1397441,1397441,6468640,1164355
4,2021/22,05/31/2022 12:00:00 AM,Mohegan Tribe On-Reservation,716607,681306,0,35301,4130,4130,31171,5611
...,...,...,...,...,...,...,...,...,...,...,...
138,2024/25,08/31/2024 12:00:00 AM,"Mohegan Digital, LLC",552906853,532307920,468730,20130203,4098131,3019530,17110672,3079921
139,2024/25,07/31/2024 12:00:00 AM,Mohegan Tribe On-Reservation,3543119,3410769,132,132219,20758,19833,112386,20229
140,2024/25,07/31/2024 12:00:00 AM,MPTN On-Reservation,3199509,3072580,265,126663,15071,15071,111592,20087
141,2024/25,07/31/2024 12:00:00 AM,"MPI Master Wagering License CT, LLC",692210181,668971512,697914,22540755,4284383,3381113,19159642,3448735


In [1]:
import pandas as pd
from scipy.stats import pearsonr

def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    """
    Devuelve una lista con las columnas numéricas del DataFrame cuya correlación absoluta con target_col 
    sea superior a umbral_corr. Si pvalue no es None, también se filtran por significación estadística.
    Si los argumentos de entrada no son adecuados, devuelve None e imprime la razón.

    Args:
        df (pd.DataFrame): DataFrame con los datos.
        target_col (str): Columna del DataFrame que será utilizada como target (debe ser numérica continua).
        umbral_corr (float): Umbral de correlación absoluta (0 a 1).
        pvalue (float, optional): Nivel de significancia para filtrar (1 - pvalue).

    Returns:
        list or None: Lista de columnas numéricas que cumplen los criterios, o None si los argumentos son inválidos.
    """
    # Comprobación 1: Que el DataFrame sea válido
    if not isinstance(df, pd.DataFrame):
        print("Error: El argumento 'df' debe ser un DataFrame de pandas.")
        return None

    # Comprobación 2: Que target_col exista y sea numérica continua
    if target_col not in df.columns:
        print(f"Error: La columna target '{target_col}' no está en el DataFrame.")
        return None
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print(f"Error: La columna target '{target_col}' no es numérica.")
        return None
    if len(df[target_col].unique()) <= 10:  # Suponemos que <10 valores únicos no es continuo
        print(f"Error: La columna target '{target_col}' no parece ser continua (tiene pocos valores únicos).")
        return None

    # Comprobación 3: Que umbral_corr sea un float entre 0 y 1
    if not isinstance(umbral_corr, (float, int)) or not (0 <= umbral_corr <= 1):
        print("Error: El argumento 'umbral_corr' debe ser un número entre 0 y 1.")
        return None

    # Comprobación 4: Que pvalue sea None o un float válido
    if pvalue is not None and (not isinstance(pvalue, (float, int)) or not (0 < pvalue <= 1)):
        print("Error: El argumento 'pvalue' debe ser None o un número entre 0 y 1.")
        return None

    # Seleccionar columnas numéricas excluyendo el target
    columnas_numericas = df.select_dtypes(include='number').columns
    columnas_numericas = [col for col in columnas_numericas if col != target_col]

    if len(columnas_numericas) == 0:
        print("Error: No hay columnas numéricas (excluyendo la target) en el DataFrame.")
        return None

    # Lista para almacenar las columnas que cumplen los criterios
    columnas_seleccionadas = []

    # Calcular correlaciones y valores p
    for col in columnas_numericas:
        correlacion, p_val = pearsonr(df[col], df[target_col])

        # Filtrar por correlación
        if abs(correlacion) > umbral_corr:
            # Filtrar por p-value si se proporciona
            if pvalue is None or p_val <= pvalue:
                columnas_seleccionadas.append(col)

    # Si no se encuentran columnas que cumplan los criterios, devolver None
    if not columnas_seleccionadas:
        print("No se encontraron columnas que cumplan con los criterios especificados.")
        return None

    return columnas_seleccionadas

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_features_cat_regression(df, target_col="", columns_seleccionadas=[], umbral_corr=0, pvalue=None):
    """
    Crea un pairplot del DataFrame considerando las columnas que cumplen con el umbral de correlación
    con la columna 'target_col' y, si se proporciona, también cumple con el test de correlación para
    el p-value.

    Args:
        df (pd.DataFrame): DataFrame con los datos.
        target_col (str): Columna de referencia, debe ser numérica (por defecto es "").
        columns_seleccionadas (list of str): Lista de columnas que cumplen con el umbral de correlación y p-value.
        umbral_corr (float): Umbral de correlación (por defecto es 0).
        pvalue (float or None): Umbral de significancia para el test de correlación (por defecto es None).

    Returns:
        list: Lista de las columnas que cumplen con los criterios de correlación y significancia.
    """

    # Si la lista de columnas seleccionadas está vacía, igualarla a las columnas numéricas del DataFrame
    if not columnas_seleccionadas:
        columnas_seleccionadas = df.select_dtypes(include='number').columns.tolist()
        # Asegurarse de que target_col esté presente si es necesario
        if target_col and target_col in columnas_numericas:
            columnas_seleccionadas.append(target_col)
        # Incluir las columnas que cumplen con el umbral de correlación
        for col in columnas_numericas:
            if col != target_col:  # Evitar target_col en la selección
                correlacion, _ = pearsonr(df[col], df[target_col])
                if abs(correlacion) > umbral_corr:
                    columns_seleccionadas.append(col)

    # Si la lista sigue vacía, imprimir un error y devolver None
    if not columnas_seleccionadas:
        print("No se encontraron columnas que cumplan con los criterios especificados.")
        return None

    # Dividir las columnas seleccionadas en grupos de máximo 5 para los pairplots
    grupos_columnas = [columnas_seleccionadas[i:i + 5] for i in range(0, len(columns_seleccionadas), 5)]

    # Generar los pairplots por grupos
    for grupo in grupos_columnas:
        sns.pairplot(df[grupo])
        plt.show()

    return columnas_seleccionadas